In [4]:
import open3d as o3d
import pyrealsense2 as rs


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [52]:


# === pytorch - yolo === #

#from IPython.display import display
from PIL import Image
import time
import torch

#config_path = "/usr/local/lib/python3.8/dist-packages/yolo/yolov3.cfg" # the YOLO net weights file
#weights_path = "/usr/local/lib/python3.8/dist-packages/yolo/yolov3.weights" # weights_path = "weights/yolov3-tiny.weights"
#labels_path = "/usr/local/lib/python3.8/dist-packages/yolo/data/coco.names"
img_path2 = "/home/physine/Documents/FYP/images/original.jpg" # crowded sence
img_path1 = "/home/physine/Documents/FYP/images/single_biker.jpg" # single biker


model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
def run_model(img):
    object_list = []
    results = model(img)
    df = results.pandas().xyxy[0]
    t0 = time.time()
    for index, row in df.iterrows():
        # xmin,   ymin,   xmax,   ymax,   confidence, objectName
        # row[0], row[1], row[2], row[3], row[4],     row[6]
        object_list.append(row)      
    print(f'infrance time of img: {time.time()-t0}')
    return object_list
    
    
def array_to_img(array):
    return Image.fromarray(array)
    
imgs = [
    "/home/physine/Documents/FYP/images/original.jpg",
    "/home/physine/Documents/FYP/images/single_biker.jpg"
]

# for img in imgs:
#     print('===========================================================')
#     objects = run_model(img)
#     for obj in objects:
#         print('-------------------------------------------------------------')
#         print(f'xmin =\t{obj[0]} \nymin =\t{obj[1]} \nxmax =\t{obj[2]} \nymax =\t{obj[3]} \nconfidence =\t{obj[4]} \nobjectName =\t{obj[6]}')


Using cache found in /home/physine/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-3 torch 1.7.1 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3011.375MB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [56]:

# === pyrealsense2 === #

import cv2
import numpy as np
from PIL import Image

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

color_path = 'V00P00A00C00_rgb.avi'
depth_path = 'V00P00A00C00_depth.avi'
colorwriter = cv2.VideoWriter(color_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (640,480), 1)
depthwriter = cv2.VideoWriter(depth_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (640,480), 1)

pipeline.start(config)

try:
    for _ in range(1):
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
             continue
        
        print(type(color_frame.get_data()))
        color_image = np.asanyarray(color_frame.get_data())

        #print(frames)
        #print(depth_frame)
        #print(color_frame.frame())

        print(type(color_image))
        
        objects = run_model(Image.fromarray(color_image))
        print(objects)
        for obj in objects:
            print('-------------------------------------------------------------')
            print(f'xmin =\t{obj[0]} \nymin =\t{obj[1]} \nxmax =\t{obj[2]} \nymax =\t{obj[3]} \nconfidence =\t{obj[4]} \nobjectName =\t{obj[6]}')
        
except Exception as e:
    print(e)
    print('error1')
    
pipeline.stop()

# try:
#     while True:     
        
#         frames = pipeline.wait_for_frames()        
#         depth_frame = frames.get_depth_frame()
#         color_frame = frames.get_color_frame()
#         if not depth_frame or not color_frame:
#             continue
        
#         #convert images to numpy arrays
#         depth_image = np.asanyarray(depth_frame.get_data())
#         color_image = np.asanyarray(color_frame.get_data())
#         depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
#         colorwriter.write(color_image)
#         depthwriter.write(depth_colormap)
        
#         cv2.imshow('Stream', color_image)
#         #cv2.imshow('Stream', depth_colormap)
        
#         if cv2.waitKey(1) == ord("q"):
#             break
# finally:
#     colorwriter.release()
#     depthwriter.release()
#     pipeline.stop()

<class 'pyrealsense2.pyrealsense2.BufData'>
<class 'numpy.ndarray'>
infrance time of img: 1.6689300537109375e-05
[]


In [108]:

# === open3d === #

import os
import open3d.ml as _ml3d
#import open3d.ml.torch as ml3d

cfg_file = "ml3d/configs/pointpillars_kitti.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model)
cfg.dataset['dataset_path'] = "/path/to/your/dataset"
dataset = ml3d.datasets.KITTI(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.ObjectDetection(model, dataset=dataset, device="gpu", **cfg.pipeline)

... 
# run inference on a single example.
result = pipeline.run_inference(data)

FileNotFoundError: File ml3d/configs/pointpillars_kitti.yml not found

Using cache found in /home/physine/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-3 torch 1.7.1 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3011.375MB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


infrance time of img: 0.0009565353393554688
-------------------------------------------------------------
xmin =	565.9375 
ymin =	425.9375 
xmax =	678.75 
ymax =	691.875 
confidence =	0.86376953125 
objectName =	person
-------------------------------------------------------------
xmin =	1066.875 
ymin =	436.875 
xmax =	1235.625 
ymax =	758.125 
confidence =	0.85888671875 
objectName =	person
-------------------------------------------------------------
xmin =	945.625 
ymin =	393.4375 
xmax =	1013.125 
ymax =	574.6875 
confidence =	0.85888671875 
objectName =	person
-------------------------------------------------------------
xmin =	21.0546875 
ymin =	621.875 
xmax =	174.0625 
ymax =	865.625 
confidence =	0.82666015625 
objectName =	bicycle
-------------------------------------------------------------
xmin =	144.375 
ymin =	443.125 
xmax =	205.625 
ymax =	660.625 
confidence =	0.8154296875 
objectName =	person
-------------------------------------------------------------
xmin =	212.812

In [ ]:
       xmin        ymin        xmax     ymax  confidence  class     name
0  146.484375  266.898438  658.203125  614.750    0.836914      1  bicycle
1  195.312500   58.304684  534.765625  505.375    0.812500      0   person